<a href="https://colab.research.google.com/github/markaaronslater/recurrent-NMT/blob/master/pos_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install stanza

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
from pos_tags import *


In [36]:
import stanza

stanza.download('de')
stanza.download('en')

src_processor = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos')
trg_processor = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')


2020-09-05 22:42:40 INFO: Downloading default packages for language: de (German)...
2020-09-05 22:42:42 INFO: File exists: /root/stanza_resources/de/default.zip.
2020-09-05 22:42:48 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-09-05 22:42:48 INFO: Downloading default packages for language: en (English)...
2020-09-05 22:42:49 INFO: File exists: /root/stanza_resources/en/default.zip.
2020-09-05 22:42:53 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-09-05 22:42:53 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |

2020-09-05 22:42:53 INFO: Use device: gpu
2020-09-05 22:42:53 INFO: Loading: tokenize
2020-09-05 22:42:53 INFO: Loading: mwt
2020-09-05 22:42:53 INFO: Loading: pos
2020-09-05 22:42:54 INFO: Done loading processors!
2020-09-05 22:42:54 WARNING: Can not find mwt: default from official model list. Ig

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
path = '/content/gdrive/My Drive/iwslt16_en_de/'

In [126]:
src_corpuses = {"train.de":'', "dev.de":'', "test.de":''}
trg_corpuses = {"train.en":'', "dev.en":''}
all_corpuses = {"src":src_corpuses, "trg":trg_corpuses}

In [127]:
load_docs2(path, all_corpuses)

In [128]:
print_corpuses2(all_corpuses)

David Gallo: Das ist Bill Lange. Ich bin Dave Gallo.
Wir werden Ihnen einige Geschichten über das Meer in Videoform erzählen.
Wir haben ein paar der unglaublichsten Aufnahmen der Titanic, die man je gesehen hat,, und wir werden Ihnen nichts davon zeigen.
Die Wahrheit ist, dass die Titanic – obwohl sie alle Kinokassenrekorde bricht – nicht gerade die aufregendste Geschichte vom Meer ist.
Ich denke, das Problem ist, dass wir das Meer für zu selbstverständlich halten.

Als ich in meinen 20ern war, hatte ich meine erste Psychotherapie-Patientin.
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley.
Sie war eine 26-jährige Frau namens Alex.
Als Alex in die erste Sitzung kam, trug sie Jeans und ein ausgebeultes Top. Sie fiel auf das Sofa in meinem Büro, schleuderte ihre Sandalen von sich und erzählte mir, sie wäre da, um über Männerprobleme zu reden.
Und als ich das hörte, war ich erleichtert.

Mein Ziel in diesem Video und dem nächsten ist es,  
einen Eindruck von der Größe d

In [129]:
process_corpuses(all_corpuses, src_processor, trg_processor, num=5)

In [130]:
print_processed_corpuses(all_corpuses)

word: David		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: :		upos: PUNCT	xpos: $.
word: Das		upos: PRON	xpos: PDS
word: ist		upos: AUX	xpos: VAFIN
word: Bill		upos: PROPN	xpos: NE
word: Lange		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################
word: Ich		upos: PRON	xpos: PPER
word: bin		upos: AUX	xpos: VAFIN
word: Dave		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################



word: Wir		upos: PRON	xpos: PPER
word: werden		upos: AUX	xpos: VAFIN
word: Ihnen		upos: PRON	xpos: PPER
word: einige		upos: PRON	xpos: PIAT
word: Geschichten		upos: NOUN	xpos: NN
word: über		upos: ADP	xpos: APPR
word: das		upos: DET	xpos: ART
word: Meer		upos: NOUN	xpos: NN
word: in		upos: ADP	xpos: APPR
word: Videoform		upos: NOUN	xpos: NN
word: erzählen		upos: VERB	xpos: VVINF
word: .		upos: PUNCT	xpos: $.
########################################



word: Wir		upos: PRON	xpos: PPER
wor

In [22]:
store_corpuses(all_corpuses, path + 'processed_corpuses')

saving processed corpuses to pickle files...
done.


In [23]:
loaded_all_corpuses = load_corpuses(path + 'processed_corpuses')

loading processed corpuses...


In [ ]:
print_processed_corpuses(loaded_all_corpuses)

In [131]:
decase_all_corpuses(all_corpuses)

In [132]:
print_processed_corpuses(all_corpuses)

word: David		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: :		upos: PUNCT	xpos: $.
word: das		upos: PRON	xpos: PDS
word: ist		upos: AUX	xpos: VAFIN
word: Bill		upos: PROPN	xpos: NE
word: Lange		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################
word: ich		upos: PRON	xpos: PPER
word: bin		upos: AUX	xpos: VAFIN
word: Dave		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################



word: wir		upos: PRON	xpos: PPER
word: werden		upos: AUX	xpos: VAFIN
word: Ihnen		upos: PRON	xpos: PPER
word: einige		upos: PRON	xpos: PIAT
word: Geschichten		upos: NOUN	xpos: NN
word: über		upos: ADP	xpos: APPR
word: das		upos: DET	xpos: ART
word: Meer		upos: NOUN	xpos: NN
word: in		upos: ADP	xpos: APPR
word: Videoform		upos: NOUN	xpos: NN
word: erzählen		upos: VERB	xpos: VVINF
word: .		upos: PUNCT	xpos: $.
########################################



word: wir		upos: PRON	xpos: PPER
wor

In [124]:
decase(all_corpuses["src"]["train.de"], num=1)

0 David
0 David

1 Gallo
1 Gallo

2 :
2 :

3 Das
3 das

4 ist
4 ist

5 Bill
5 Bill

6 Lange
6 Lange

7 .
7 .

0 Ich
0 ich

1 bin
1 bin

2 Dave
2 Dave

3 Gallo
3 Gallo

4 .
4 .



In [103]:
print_processed_corpus(all_corpuses["src"]["train.de"], num=1)

word: David		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: :		upos: PUNCT	xpos: $.
word: ist		upos: PRON	xpos: PDS
word: ist		upos: AUX	xpos: VAFIN
word: Bill		upos: PROPN	xpos: NE
word: Lange		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################
word: ich		upos: PRON	xpos: PPER
word: bin		upos: AUX	xpos: VAFIN
word: Dave		upos: PROPN	xpos: NE
word: Gallo		upos: PROPN	xpos: NE
word: .		upos: PUNCT	xpos: $.
########################################





In [24]:
# with open(path + 'train.en', mode='rt', encoding='utf-8') as f:
#   train_en = f.read().strip().split('\n')

# with open(path + 'train.de', mode='rt', encoding='utf-8') as f:
#   train_de = f.read().strip().split('\n')

In [25]:
# num = 5
# for sent in train_en[:num]:
#   print(sent)
# print()
# num = 5
# for sent in train_de[:num]:
#   print(sent)


David Gallo: This is Bill Lange. I'm Dave Gallo.
And we're going to tell you some stories from the sea here in video.
We've got some of the most incredible video of Titanic that's ever been seen, and we're not going to show you any of it.
The truth of the matter is that the Titanic -- even though it's breaking all sorts of box office records -- it's not the most exciting story from the sea.
And the problem, I think, is that we take the ocean for granted.

David Gallo: Das ist Bill Lange. Ich bin Dave Gallo.
Wir werden Ihnen einige Geschichten über das Meer in Videoform erzählen.
Wir haben ein paar der unglaublichsten Aufnahmen der Titanic, die man je gesehen hat,, und wir werden Ihnen nichts davon zeigen.
Die Wahrheit ist, dass die Titanic – obwohl sie alle Kinokassenrekorde bricht – nicht gerade die aufregendste Geschichte vom Meer ist.
Ich denke, das Problem ist, dass wir das Meer für zu selbstverständlich halten.


In [26]:
# for sentence in train_en[:5]:
#   doc = nlp_en(sentence)
#   for sent in doc.sentences:
#     for word in sent.words: 
#       print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
#     print()
#   print()
#   print()
#   print()


word: David		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Gallo		upos: PROPN	xpos: NNP	feats: Number=Sing
word: :		upos: PUNCT	xpos: :	feats: _

word: This		upos: PRON	xpos: DT	feats: Number=Sing|PronType=Dem
word: is		upos: AUX	xpos: VBZ	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: Bill		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Lange		upos: PROPN	xpos: NNP	feats: Number=Sing
word: .		upos: PUNCT	xpos: .	feats: _

word: I		upos: PRON	xpos: PRP	feats: Case=Nom|Number=Sing|Person=1|PronType=Prs
word: 'm		upos: AUX	xpos: VBP	feats: Mood=Ind|Tense=Pres|VerbForm=Fin
word: Dave		upos: PROPN	xpos: NNP	feats: Number=Sing
word: Gallo		upos: PROPN	xpos: NNP	feats: Number=Sing
word: .		upos: PUNCT	xpos: .	feats: _




word: And		upos: CCONJ	xpos: CC	feats: _
word: we		upos: PRON	xpos: PRP	feats: Case=Nom|Number=Plur|Person=1|PronType=Prs
word: 're		upos: AUX	xpos: VBP	feats: Mood=Ind|Tense=Pres|VerbForm=Fin
word: going		upos: VERB	xpos: VBG	feats: Tense=Pres|VerbForm

In [27]:
s = 'And we said, "Well what about play and recess?"'
doc = trg_processor(s)
for sent in doc.sentences:
  for word in sent.words: 
    print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
  print()

word: And		upos: CCONJ	xpos: CC	feats: _
word: we		upos: PRON	xpos: PRP	feats: Case=Nom|Number=Plur|Person=1|PronType=Prs
word: said		upos: VERB	xpos: VBD	feats: Mood=Ind|Tense=Past|VerbForm=Fin
word: ,		upos: PUNCT	xpos: ,	feats: _
word: "		upos: PUNCT	xpos: ``	feats: _
word: Well		upos: INTJ	xpos: UH	feats: _
word: what		upos: PRON	xpos: WP	feats: PronType=Int
word: about		upos: ADP	xpos: IN	feats: _
word: play		upos: NOUN	xpos: NN	feats: Number=Sing
word: and		upos: CCONJ	xpos: CC	feats: _
word: recess		upos: NOUN	xpos: NN	feats: Number=Sing
word: ?		upos: PUNCT	xpos: .	feats: _
word: "		upos: PUNCT	xpos: ''	feats: _



In [139]:
s = 'Sie benutzen die Biolumineszenz, um Geschlechtspartner anzulocken, Beute zu ködern und zur Verständigung.'
doc = src_processor(s)
for sent in doc.sentences:
  for word in sent.words: 
    print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
  print()

word: Sie		upos: PRON	xpos: PPER	feats: Case=Nom|Number=Plur|Person=3|PronType=Prs
word: benutzen		upos: VERB	xpos: VVFIN	feats: Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin
word: die		upos: DET	xpos: ART	feats: Case=Acc|Definite=Def|Gender=Fem|Number=Sing|PronType=Art
word: Biolumineszenz		upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Sing
word: ,		upos: PUNCT	xpos: $,	feats: _
word: um		upos: ADP	xpos: KOUI	feats: _
word: Geschlechtspartner		upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Masc|Number=Plur
word: anzulocken		upos: VERB	xpos: VVIZU	feats: VerbForm=Inf
word: ,		upos: PUNCT	xpos: $,	feats: _
word: Beute		upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Sing
word: zu		upos: PART	xpos: PTKZU	feats: _
word: ködern		upos: VERB	xpos: VVINF	feats: VerbForm=Inf
word: und		upos: CCONJ	xpos: KON	feats: _
word: zu		upos: ADP	xpos: APPR	feats: _
word: der		upos: DET	xpos: ART	feats: Case=Dat|Definite=Def|Gender=Fem|Number=Sing|PronType=Art
word: Verständigung		upos:

In [27]:
# for sentence in train_de[:5]:
#   doc = nlp_de(sentence)
#   for sent in doc.sentences:
#     for word in sent.words: 
#       print(f'word: {word.text}\t\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}')
#     print()
#   print()
#   print()
#   print()

word: David		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Gallo		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: :		upos: PUNCT	xpos: $.	feats: _
word: Das		upos: PRON	xpos: PDS	feats: Case=Nom|Gender=Neut|Number=Sing|PronType=Dem
word: ist		upos: AUX	xpos: VAFIN	feats: Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
word: Bill		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Lange		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: .		upos: PUNCT	xpos: $.	feats: _

word: Ich		upos: PRON	xpos: PPER	feats: Case=Nom|Number=Sing|Person=1|PronType=Prs
word: bin		upos: AUX	xpos: VAFIN	feats: Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin
word: Dave		upos: PROPN	xpos: NE	feats: Case=Nom|Gender=Masc|Number=Sing
word: Gallo		upos: PROPN	xpos: NE	feats: Case=Acc|Gender=Masc|Number=Sing
word: .		upos: PUNCT	xpos: $.	feats: _




word: Wir		upos: PRON	xpos: PPER	feats: Case=Nom|Number=Plur|Person=1|PronType=P